# CiRA Demonstration

This notebook demonstrates the usage of the CiRA pipeline. Be sure to fulfill all prerequisites as specified in the [readme](README.md), especially the availability of the two pretrained models, both of which are available for download on Zenodo (find the classifier [here](https://zenodo.org/record/5159501#.Ytq28ITP3-g) and the labeler [here](https://zenodo.org/record/5550387#.Ytq3QYTP3-g) (use the model named `roberta_dropout_linear_layer_multilabel.ckpt` for optimal performance)). If you run the code from the provided container, the models are already included.

## Locate the Models

The location of the two pre-trained models must be available to the CiRA converter. You can hardcode the location of the pre-trained models to the two variables `model_classification` and `model_labeling`, or use the subsequent snippet, in which the location of the models is determined. If you run the code locally, 

1. create a `.env` file and 
2. define the variables `MODEL_CLASSIFICATION` and `MODEL_LABELING` with the respective URLs. 

If you run the code from the container, then the suffix `_DEV` in the environment variables will distinguish the location of the models locally from those in the container.

In [1]:
import os, dotenv
dotenv.load_dotenv()

model_env_suffix = '_DEV' if ('DEV_CONTAINER' in os.environ) else ''

model_classification = os.environ[f'MODEL_CLASSIFICATION{model_env_suffix}']
model_labeling = os.environ[f'MODEL_LABELING{model_env_suffix}']

## Create the Converter

Instantiate the CiRA converter which is realized in [src.cira.CiRAConverter](./src/cira.py). The instantiation of the converter will output some warning messages, which can be safely ignored.

In [2]:
from src.cira import CiRAConverter
cira = CiRAConverter(classifier_causal_model_path=model_classification, converter_s2l_model_path=model_labeling)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 855kB/s] 
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 11.2kB/s]
Downloading: 100%|██████████| 436k/436k [00:01<00:00, 226kB/s] 
Downloading: 100%|██████████| 570/570 [00:00<00:00, 455kB/s]
Downloading: 100%|██████████| 436M/436M [00:30<00:00, 14.2MB/s] 
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing Bert

## Apply the converter

Finally, apply the converter to utilize the CiRA pipeline. This usually includes four steps:

1. Define a single natural language sentence. CiRA is currently only trained on grammatically correct, full sentences.
2. Classify the sentence to determine, whether the sentence is causal or nor.
3. If the sentence is causal, process it to generate labels, convert it into a cause-effect-graph, and finally derive a minimal set of test cases covering the requirement.

Non-causal sentences cannot be processed by the pipeline.

In [3]:
sentence: str = "If the red button is pressed and background operations are not running then the system shuts down."

In [4]:
causal, confidence = cira.classify(sentence)
print(f'CiRA classified the sentence "{sentence}" to be {"*" if causal else "*non"}causal* with a confidence of {confidence:.2%}.')

CiRA classified the sentence "If the red button is pressed and background operations are not running then the system shuts down." to be *causal* with a confidence of 97.73%.


In [5]:
if causal:
    labels, graph, suite = cira.process(sentence)

    print(f'Labels associated to the sentence: {labels}\n')
    print(f'Converted cause-effect-graph: {graph}\n')
    print(f'Minimal test suite: \n{suite}')

Labels associated to the sentence: [[3> (L0) Cause1  (AND L1): [3> (L4) Variable <17] [18> (L7) Condition <28] <28], [33> (L1) Cause2  (None L2): [33> (L5) Variable <54] [63> (L8) Condition <70] [55> (L10) Negation <62] <70], [76> (L2) Effect1 : [76> (L6) Variable <86] [87> (L9) Condition <97] <97], [29> (L3) Conjunction <32], [3> (L4) Variable <17], [33> (L5) Variable <54], [76> (L6) Variable <86], [18> (L7) Condition <28], [63> (L8) Condition <70], [87> (L9) Condition <97], [55> (L10) Negation <62]]

Converted cause-effect-graph: ([the red button].(is pressed) && NOT [background operations].(running)) ===> [the system].(shuts down)

Minimal test suite: 
  id  | the red button    background operations    | the system
----  ------------------  -----------------------  ----------------
   1  | is pressed        not running              | shuts down
   2  | not is pressed    not running              | not shuts down
   3  | is pressed        running                  | not shuts down
